In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [21]:
#insert a webpage and scrap it into a df
def webscrap_feldheim(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    data = soup.findAll("div", {"class": "des product details product-item-details"})    
    info = [[x.text for x in y.findAll('div')] for y in data]
    df = pd.DataFrame(info) 
    rating_df=df[df[0].str.contains('Rating')]
    new_df=df[~df[0].str.contains('Rating')]
    rating_df=rating_df.iloc[:, 0:8]
    rating_df["reviews"] = rating_df[3].str.split(' ').str[0]
    rating_df["ratings"]= rating_df[2]
    rating_df = rating_df.drop([0,1,2,3], axis=1)
    rating_df.rename(columns={4: "title", 5: "price", 6: "delete1", 7: "in_stock"}, inplace=True)    
    new_df = new_df.iloc[:, 0:6]
    new_df.rename(columns={0: "title", 1: "price", 2: "delete1", 3: "in_stock", 4:"reviews", 5:"ratings"}, inplace=True)
    new_df["reviews"]=None  
    new_df["ratings"]=None
    clean_df=pd.concat([new_df, rating_df])
    clean_df=clean_df.drop(["delete1"], axis=1)
    clean_df=clean_df[~clean_df["title"].str.contains('Hebrew')]
    clean_df=clean_df[~clean_df["title"].str.contains('Yiddish')]
    clean_df=clean_df[~clean_df["title"].str.contains('Spanish')]
    clean_df=clean_df[~clean_df["title"].str.contains('French')]
    clean_df=clean_df[~clean_df["title"].str.contains('Russian')]
    clean_df=clean_df[~clean_df["title"].str.contains('Portuguese')]
    clean_df["in_stock"] = clean_df["in_stock"].str.strip()
    clean_df["in_stock"] = clean_df["in_stock"].str.replace("In stock  WishList", "True").str.replace("Add to Cart", "True")
    clean_df['in_stock'] = clean_df['in_stock'].str.replace('Out of stock  WishList', 'False').str.replace('Add to Cart', 'False').str.replace('Out of stock', 'False')
    category = re.search(r'm/(.*).html', url)
    category=category.group(1)
    clean_df["category"]=category
    clean_df["publisher"]="feldheim"
    return clean_df

In [22]:
history= webscrap_feldheim('https://www.feldheim.com/history.html?product_list_limit=all')
literature= webscrap_feldheim('https://www.feldheim.com/contemporary-literature.html?product_list_limit=all')
jewish= webscrap_feldheim('https://www.feldheim.com/torah/jewish-thought-outlook.html?product_list_limit=all')

In [24]:
feldheim=pd.concat([history, literature, jewish], axis=0)

In [25]:
feldheim.drop_duplicates(subset=['title'], keep='first', inplace=True, ignore_index=True)

In [26]:
feldheim.to_csv('feldheim.csv')